# Python中时间格式的转换和计算


## Python 表示时间有以下格式：
### 1. Pandas 的datetime64
datetime64是Pandas从excel读取时间单元格默认的存储格式，这个格式不支持提取年月日，打印出来是这样的：

    
```[python]
>>>dt64 = np.datetime64('1970-01-01T00:00:00Z')
>>>print(dt64)
1970-01-01T00:00:00.00
```
上面的例子中时间只给到秒的精度，实际上datetime64 最高精度存储到纳秒(ns), 如果是从excel读取出来的，则默认会补全到纳秒精度，打印会显示
```
1970-01-01T00:00:00.00.000000000
```
datetime64采用UTC时区
### 2. Python 的datetime
datetime就是python自带的时间存储格式,打印出来这样：
```[python]
1970-01-01 00:00:00
```
### 3.Timestamp 即时间戳
Timestamp 存储的是时间T 和基准时间的间隔，以秒s为单位
(注意如果是datetime64定义的timestamp,则以纳秒ns为单位)

## 具体的例子：
### datetime64 格式：

In [1]:
import numpy as np
import datetime
import time

In [2]:
# 定义并查看dt64格式时间
dt64 = np.datetime64('2019-09-01T00:00:00.000000000')
print(dt64)
print(type(dt64))

2019-09-01T00:00:00.000000000
<class 'numpy.datetime64'>


In [3]:
# 注意， dt64格式最多精确到10^{-9}秒，也就是纳秒，在T00:00:00. 后面最多9个0，少于9位只是精度降低，多于9位则会溢出
# 看下面的例子： 
# 省略秒后面的所有0， 时间精确到秒：
dt64 = np.datetime64('2019-09-01T00:00:00')
print(dt64)

2019-09-01T00:00:00


In [4]:
# 位数过多的溢出, 则会得到不正确的结果
dt64 = np.datetime64('2019-09-01T00:00:00.0000000000')
print(dt64)

1969-12-24T03:52:33.734688112640


### 时间戳

In [5]:
# 查看时间戳，时间戳为与1970年01月01日0:00:00时刻的时间差，以秒为单位
ts = time.time()
print(ts)
print(type(ts))

1571636931.391643
<class 'float'>


### datetime 格式

In [6]:
# 查看dt格式时间， dt格式时间包括9份信息，本别是： year, mon, day, hour, min, sec, 星期(0-6, 0 代表周一)， 当前年的第多少天（1-366）， 是否夏令时 
dt = time.localtime(time.time())
print(dt)
print(type(dt))

time.struct_time(tm_year=2019, tm_mon=10, tm_mday=21, tm_hour=13, tm_min=48, tm_sec=51, tm_wday=0, tm_yday=294, tm_isdst=0)
<class 'time.struct_time'>


In [7]:
# 转化为格式化时间并输出
print(time.strftime('%Y-%m-%d %H:%M:%S', dt))

2019-10-21 13:48:51


In [8]:
# 也可以直接定义格式化时间，存储为格式字符串“str”
dt_str = time.strftime('%Y-%m-%d %H:%M:%S', dt)
print(dt_str)
print (type(dt_str))

2019-10-21 13:48:51
<class 'str'>


### 格式转换

In [9]:
# 格式字符串（str）时间转换为 dt64
dt64_c = np.datetime64(dt_str)
print(dt64_c)
print(type(dt64_c))

2019-10-21T13:48:51
<class 'numpy.datetime64'>


In [10]:
# dt64 转换回格式字符串时间
dt_c = dt64_c.astype(datetime.datetime)
print(dt_c)
print(type(dt_c))

2019-10-21 13:48:51
<class 'datetime.datetime'>


### 计算时间差（以天数差为例， 可以用来做倒计时器）：

In [11]:
# 计算天数差
# 例如，我们要计算2020年8月5日距离现在的天数
target_day_dt64 = np.datetime64('2025-08-05T00:00:00')
now_date_str = time.strftime('%Y-%m-%d', time.localtime(time.time()))
# 先计算时间戳差
ts_delta = target_day_dt64 - np.datetime64(now_date_str)
print(ts_delta)
print(type(ts_delta))

182736000 seconds
<class 'numpy.timedelta64'>


In [12]:
# 再将时间戳差转换为天数差（除以np.timedelta64， 得到的结果是无单位的，对比下面cell的验证，直接除以一天的秒数，得到的还是带单位的数）
day_delta = ts_delta/np.timedelta64(1,'D')
print('目标日期距离今天 %s 天' % day_delta)
print(type(day_delta))


目标日期距离今天 2115.0 天
<class 'numpy.float64'>


In [13]:
## 验证一下： （只供验证天数是否吻合，这里的单位还是“秒”，单位是不对的）
ts_delta/60/60/24


numpy.timedelta64(2115,'s')